# Upload Measurement Data and run Analysis

Now you will post your measurement data and analysis to the database via the API.

**If you are running the tutorials in DoLab, the following instructions are not necessary and you can skip directly to the next cell.**

You will need to authenticate to the database with your username and password. To make this easy,  you can create a file called `.env` in this folder and complete it with your organization's URL and authentication information as follows:

```bash
dodata_url=https://animal.doplaydo.com
dodata_user=demo
dodata_password=yours
dodata_db=animal.dodata.db.doplaydo.com
dodata_db_user=full_access
dodata_db_password=yours

```

If you haven't defined a `.env` file or saved your credentials to your environment variables, you will be prompted for your credentials now.

In [ ]:
import doplaydo.dodata as dd
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import requests
import getpass

username = getpass.getuser()

Let's now create a project. 

In normal circumstances, everyone will be sharing and contributing to a project. In this demo, however, we want to *keep your project separate* from other users for clarity, so we will append your username to the project name. This way you can also safely delete and recreate projects without creating issues for others. If you prefer though, you can change the `PROJECT_NAME` to anything you like. Just be sure to update it in the subsequent notebooks of this tutorial as well.

In [ ]:
PROJECT_NAME = f"rings-{username}"
MEASUREMENTS_PATH = Path("6d4c615ff105/")

If you've been running this demo multiple times, you might already have one with the same name. Let's delete it so you can start fresh.

In [ ]:
dd.project.delete(project_name=PROJECT_NAME).text

## New project

You can create the project, upload the design manifest, and upload the wafer definitions through the Webapp as well as programmatically using this notebook

### Upload Project

You can create a new project and extract all cells & devices below for the `RidgeLoss` and `RibLoss`

The expressions are regex expressions. For intro and testing your regexes you can check out [regex101](https://regex101.com)

To only extract top cells set `max_hierarchy_lvl=-1` and `min_hierarchy_lvl=-1`

To disable extraction use a max_hierarchy_lvl < min_hierarchy_lvl

Whitelists take precedence over blacklists, so if you define both, it uses only the whitelist.

In [ ]:
cell_extraction = [
    dd.project.Extraction(
        cell_name="rings",
        cell_white_list=["^RingDouble"],
        min_hierarchy_lvl=0,
        max_hierarchy_lvl=0,
    )
]

dd.project.create(
    project_name=PROJECT_NAME,
    eda_file="test_chip.gds",
    lyp_file="generic.lyp",
    cell_extractions=cell_extraction,
).text

### Upload Design Manifest

The design manifest is a CSV file that includes all the cell names, the cell settings, a list of analysis to trigger, and a list of settings for each analysis.

In [ ]:
dm = pd.read_csv("design_manifest.csv")
dm

In [ ]:
dd.project.upload_design_manifest(
    project_name=PROJECT_NAME, filepath="design_manifest.csv"
).text

In [ ]:
dd.project.download_design_manifest(
    project_name=PROJECT_NAME, filepath="design_manifest_downloaded.csv"
)

### Upload Wafer Definitions

The wafer definition is a JSON file where you can define the wafer names and die names and location for each wafer.

In [ ]:
dd.project.upload_wafer_definitions(
    project_name=PROJECT_NAME, filepath="wafer_definitions.json"
).text

## Upload data

Your Tester can output the data in JSON files. It does not need to be Python.

You can get all paths which have measurement data within the test path.

In [ ]:
data_files = list(MEASUREMENTS_PATH.glob("**/data.json"))
print(data_files[0].parts)

 You should define a plotting per measurement type in python. Your plots can evolve over time even for the same measurement type.

Required:

```yaml
- x_name (str): x-axis name
- y_name (str): y-axis name
- x_col (str): x-column to plot
- y_col (str or list[str]): y-column(s) to plot; can be multiple
```


Optional:
```yaml

- scatter (bool): whether to plot as scatter as opposed to line traces
- x_units (str): x-axis units
- y_units (str): y-axis units
- x_log_axis (bool): whether to plot the x-axis on log scale
- y_log_axis (bool): whether to plot the y-axis on log scale
- x_limits (list[int, int]): clip x-axis data using these limits as bounds (example: [10, 100])
- y_limits (list[int, int]): clip y-axis data using these limits as bounds (example: [20, 100])
- sort_by (dict[str, bool]): columns to sort data before plotting. Boolean specifies whether to sort each column in ascending order.
                             (example: {"wavelegths": True, "optical_power": False})
- grouping (dict[str, int]): columns to group data before plotting. Integer specifies decimal places to round each column.
                             Different series will be plotted for unique combinations of x column, y column(s), and rounded column values.
                             (example: {"port": 1, "attenuation": 2})

```

In [ ]:
spectrum_measurement_type = dd.api.device_data.PlottingKwargs(
    x_name="wavelength",
    y_name="output_power",
    x_col="wavelength",
    # y_col=["output_power"],  # can also be a string for a single value
    y_col="output_power",
)

### Upload measurements

You can now upload measurement data.

This is a bare bones example, in a production setting, you can also add validation, logging, and error handling to ensure a smooth operation.

Every measurement you upload will trigger all the analysis that you defined in the design manifest.

In [ ]:
NUMBER_OF_THREADS = 12
wafer_set = set()
die_set = set()

In [ ]:
if NUMBER_OF_THREADS == 1:
    for path in tqdm(data_files):
        wafer_name = path.parts[0]
        die_x, die_y = path.parts[1].split("_")

        r = dd.api.device_data.upload(
            file=path,
            project_name=PROJECT_NAME,
            wafer_name=wafer_name,
            die_x=die_x,
            die_y=die_y,
            device_name=path.parts[2],
            data_type="simulation",
            plotting_kwargs=spectrum_measurement_type,
        )
        wafer_set.add(wafer_name)
        die_set.add(path.parts[2])
        r.raise_for_status()

In [ ]:
data_files = list(MEASUREMENTS_PATH.glob("**/data.json"))
project_names = []
device_names = []
die_names = []
die_xs = []
die_ys = []
wafer_names = []
plotting_kwargs = []
data_types = []

for path in data_files:
    device_name = path.parts[2]
    die_name = path.parts[1]
    die_x, die_y = die_name.split("_")
    wafer_name = path.parts[0]

    device_names.append(device_name)
    die_names.append(die_name)
    die_xs.append(die_x)
    die_ys.append(die_y)
    wafer_names.append(wafer_name)
    plotting_kwargs.append(spectrum_measurement_type)
    project_names.append(PROJECT_NAME)
    data_types.append("measurement")

In [ ]:
if NUMBER_OF_THREADS > 1:
    dd.device_data.upload_multi(
        files=data_files,
        project_names=project_names,
        wafer_names=wafer_names,
        die_xs=die_xs,
        die_ys=die_ys,
        device_names=device_names,
        data_types=data_types,
        plotting_kwargs=plotting_kwargs,
        progress_bar=True,
    )

In [ ]:
die_names_set = set(die_names)
wafer_set = set(wafer_names)

print(die_names_set)
print(wafer_set)

In [ ]:
plots = dd.analysis.get_device_data_analysis_plots(
    project_name=PROJECT_NAME,
    # device_name="rings_RingDouble-20-200-_add_fiber_array_ea36ac06_331580_243771",
    device_name=device_names[0],
    die_x=0,
    die_y=0,
    wafer_name="6d4c615ff105",
)
plots

In [ ]:
for plot in plots:
    display(plot)